In [1]:
import dotenv
import google.generativeai as genai
import os

dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set")


genai.configure(api_key=GEMINI_API_KEY)

c:\Users\Waadl\Documents\aadildev\projects\etsy-automation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model = genai.GenerativeModel('gemini-pro')

In [20]:
title_generation_config = {
  "temperature": 0.1,
  "max_output_tokens": 20,
  "top_k": 1,
  "response_mime_type": "text/plain",
}

description_generation_config = genai.types.GenerationConfig(
  max_output_tokens=500,
  stop_sequences=["Keywords:"]
)

In [21]:
def create_description(filename: str):
    text_prompt = """I will give you a filename representing a piano sheet music pdf, 
    and your task is to create an etsy listing description that adequately talks about the pdf. 
    Subtly disperse keywords for SEO throughout the description text, 
    writing them with natural language."""

    response = model.generate_content(contents=[text_prompt, filename], 
                                      generation_config=description_generation_config)
    return response.text


In [22]:
def create_sheet_name(filename: str):
    text_prompt = """I will give you a filename representing a sheet music pdf, 
    and your task is to extract the readable name formatted with spaces. 
    Do not output anything else except JSON in the form like so: {"name": "name here"}

    For example, the filename 7_Rings_-_Ariana_Grande__Piano_Tutorial.pdf should return a name like 
    "7 Rings - Ariana Grande Piano Tutorial" and You_re Not Alone FF9.pdf should return a name like 
    "You're Not Alone - FF9". 
    Use your judgment when creating the best possible title names from the filenames 
    as I am creating these for customers.
    """

    response = model.generate_content(contents=[text_prompt, filename], generation_config=title_generation_config)
    return response.text

In [15]:
create_sheet_name("Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf")

{"name": "Avengers Theme - Infinity War Arrangement Piano Tutorial"}


In [23]:
print(create_description("Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf"))

**Unleash the Epic Power of the Infinity Gauntlet with this Piano Sheet Music**

Prepare to take on the musical prowess of the legendary Avengers with this exclusive piano sheet music arrangement for "Avengers: Infinity War." Experience the thrilling journey of Earth's Mightiest Heroes as they confront the ultimate cosmic threat.

**Awaken Your Inner Superhero with Precise Notation**

This meticulously crafted PDF provides crystal-clear notation, enabling you to effortlessly navigate complex passages and grasp the true essence of the score. Whether you're a seasoned maestro or an aspiring prodigy, this sheet music will empower you to unleash your musical potential.

**Immerse Yourself in the Realm of Marvel**

As you traverse each measure, feel the weight of Thanos's imposing gauntlet and the unwavering determination of the Avengers. This arrangement captures the cinematic grandeur of the film, transporting you to the heart of the epic battle.

**Master the Symphony of Heroism**

With 

2000_Miles_Piano_Cover_Sheet_Music.pdf
365_-__Katy_Perry__Zedd__Piano_Tutorial_.pdf
7_Rings_-_Ariana_Grande__Piano_Tutorial.pdf
Aeris_ Theme FF7.pdf
At the South Gate Border (MoL) FF9.pdf
Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf
A_Spaceman_Came_Travelling_Piano_Tutorial_.pdf
Bates_Motel_Piano_Cover_Sheet_Music.pdf
Battle with Gilgamesh FF5.pdf
Big_Little_Lies_-_Main_theme_Music_Piano_Version.pdf
BoJack_Horseman_Main_Theme__Piano_Tutorial.mscz.pdf
Bran Bal FF9.pdf
break_up_with_your_girlfriend_im_bored_-_Ariana_Grande__Piano_Tutorial.pdf
California_Dreamin_-_The_Mamas_And_The_Papas__Piano_Tutorial.pdf
Camila_Cabello_ft_Young_Thug__-_Havana_Piano_Sheet_Music.pdf
Cant_Help_Falling_In_Love__Crazy_Rich_Asians__Piano_Cover_Sheet_Music.mscz.pdf
Chemistry - Period.pdf
Childish_Gambino_-_This_Is_America__Piano_Cover__Sheet_Music.pdf
Chopin - Nocturne-op9-no1.pdf
Chopin - Nocturne-op9-no2.pdf
Chopin - Regentropfen Prelude Op. 28 Nr.15.pdf
Chopin - Valse-op-64-n-2.pdf
Chopin 

# Create thumbnail

In [45]:
from PIL import Image
import os
import fitz
import re

def create_thumbnail(filename: str):
    THUMBNAIL_SIZE = [570, 456]
    THUMBNAIL_SHEET_MUSIC_HEIGHT = 422

    # 1. extract the first page of the pdf as an image
    filepath = f"sheetmusic/{filename}"
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File {filename} not found")
    pdf = fitz.open(filepath)
    pdf_first_page = next(pdf.pages())
    pix = pdf_first_page.get_pixmap()

    img_name = re.split(r"\.", filename)[0]

    pdf_image_path = f"tempimages/{img_name}.jpg"
    pix.save(pdf_image_path)

    # 2. create a thumbnail image with the sheet music image
    background_image = Image.open("thumbnail-background.jpg")
    pdf_image = Image.open(pdf_image_path)

        # max height of 536, maintains aspect ratio
    pdf_image.thumbnail((THUMBNAIL_SIZE[0], THUMBNAIL_SHEET_MUSIC_HEIGHT))
    pdf_image.save(pdf_image_path)

    background_image.paste(pdf_image, (136, 17))
    background_image.save(f"thumbnails/{img_name}.jpg")

    os.remove(pdf_image_path)
    return f"thumbnails/{img_name}.jpg"
    

In [46]:
create_thumbnail("Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.pdf")

(298, 422)


'thumbnails/Avengers_Theme_-_Infinity_War_Arrangement__Piano_Tutorial.jpg'